In [ ]:
import tensorflow as tf, numpy as np, matplotlib.pyplot as plt
plt.style.use('ggplot')

## Difference Between Tensor and Python Variable

In [ ]:
var = 1.
print("python variable: ", var)
print()

var_tensor = tf.constant(1.)
print("tensor: ", var_tensor)

In [ ]:
with tf.Session() as sess:
    print("透過tensorflow session取得tensor value: ", sess.run(var_tensor))

<br/>
<br/>
<br/>
## Simple Regression Example
- Formula: $ Y = 0.5X + 0.3 $

In [ ]:
formula = lambda x: 0.5 * x + 0.3
data_x = np.linspace(0, 1, 100)
plt.plot(data_x, formula(data_x))
plt.show()

In [ ]:
import tensorflow as tf, numpy as np

n_epoch = 200
learning_rate = 0.5

outerX = np.random.random(10000)
outerY = 0.5 * outerX + 0.3
outerX, outerY

model_dir = "./model/linear"

tf.reset_default_graph()
g = tf.Graph()
with g.as_default():
    with tf.variable_scope("inputs"):
        # 接收 outerX data
        placeholderX = tf.placeholder(tf.float32, shape=[None], name="placeholder_x")
        # 接收 outerY data
        placeholderY = tf.placeholder(tf.float32, shape=[None], name="placeholder_y")
        
    with tf.variable_scope("formula"):
        varW = tf.Variable(tf.random_uniform(shape=[1]), tf.float32, name="var_w")
        varB = tf.Variable(tf.random_uniform(shape=[1]), tf.float32, name="var_b")
        # 公式 0.5X + 0.3, 期望varW會慢慢逼近0.5, varB會慢慢逼近0.3
        varY = varW * placeholderX + varB
        # tensorboard weights logs
        tf.summary.histogram("varW", varW)
        tf.summary.histogram("varB", varB)
    # mean square error
    loss = tf.losses.mean_squared_error(placeholderY, varY)
    # tensorboard weights logs
    tf.summary.scalar("loss", loss)
    # 使用 GradientDescentOptimizer
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    # 最小化 loss, 此時會 backpropagation 去調整 varW, varB的值
    trainer = optimizer.minimize(loss)
    merge = tf.summary.merge_all()
    
    with tf.Session(graph=g) as sess:
        w = tf.summary.FileWriter("./model/linear", sess.graph)
        init = tf.global_variables_initializer()
        sess.run(init)
        for i in range(n_epoch):
            _, w_, b_ = sess.run([trainer, varW, varB], feed_dict={placeholderX: outerX, placeholderY: outerY})
            print('step: {} varW: {}, varB: {}'.format(i, w_, b_) )
            if (i + 1) % 20 == 0:
                merge_ = sess.run(merge, feed_dict={placeholderX: outerX, placeholderY: outerY})
                w.add_summary(merge_, i)

## Check Tensorboard

In [ ]:
from google.datalab.ml import TensorBoard

tb = TensorBoard()
# stop prev built tensorboard pid
for _, r in tb.list().iterrows(): tb.stop(r.pid)

tb.start(model_dir)
tb.list()

## Stop Tensorboard

In [ ]:
for _, r in TensorBoard().list().iterrows(): tb.stop(r.pid)

<br/>
<br/>
<br/>
<br/>
<br/>
## Model Save and Load

In [ ]:
import tensorflow as tf, numpy as np

class LinearModel(object):
    def __init__(self):
        """Init graph"""
        tf.reset_default_graph()
        self.graph = tf.Graph()
        with self.graph.as_default():
            with tf.variable_scope("inputs"):
                # 接收 outerX data
                self.placeholderX = tf.placeholder(tf.float32, shape=[None], name="placeholder_x")
                # 接收 outerY data
                self.placeholderY = tf.placeholder(tf.float32, shape=[None], name="placeholder_y")

            with tf.variable_scope("formula"):
                self.varW = tf.Variable(tf.random_uniform(shape=[1]), tf.float32, name="var_w")
                self.varB = tf.Variable(tf.random_uniform(shape=[1]), tf.float32, name="var_b")
                self.varY = self.varW * self.placeholderX + self.varB
                # tensorboard weights logs
                tf.summary.histogram("varW", self.varW)
                tf.summary.histogram("varB", self.varB)
            # 均方誤差
            self.loss = tf.losses.mean_squared_error(self.placeholderY, self.varY)
            # tensorboard weights logs
            tf.summary.scalar("loss", self.loss)
            # 使用 AdamOptimizer
            self.optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
            # 最小化 loss, 此時會 backpropagation 去調整 varW, varB的值
            self.trainer = self.optimizer.minimize(self.loss)
            self.merge = tf.summary.merge_all()
            # 儲存物件
            self.saver = tf.train.Saver(tf.global_variables())
            
    def load_weight(self, sess, model_dir):
        """load latest saved model"""
        latestCkpt = tf.train.latest_checkpoint(model_dir)
        if latestCkpt:
            self.saver.restore(sess, latestCkpt)
            print("load weight success!")
        return latestCkpt

## Save Model

In [ ]:
n_epoch = 200
learning_rate = 0.5

outerX = np.random.random(10000)
outerY = 0.5 * outerX + 0.3
outerX, outerY

model_dir = "./model/linear"

model = LinearModel()
with tf.Session(graph=model.graph) as sess:
    w = tf.summary.FileWriter(model_dir, sess.graph)
    init = tf.global_variables_initializer()
    sess.run(init)
    for i in range(n_epoch):
        _, w_, b_ = sess.run([model.trainer, model.varW, model.varB], feed_dict={model.placeholderX: outerX, model.placeholderY: outerY})
        print('step: {} varW: {}, varB: {}'.format(i, w_, b_) )
        if (i + 1) % 50 == 0:
            merge_ = sess.run(model.merge, feed_dict={model.placeholderX: outerX, model.placeholderY: outerY})
            w.add_summary(merge_, i)
            
        if (i + 1) % 50 == 0:
            ''' Save model '''
            model.saver.save(sess, save_path=model_dir + "/linear_model", global_step=i)

## Load Model

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

model_dir = "./model/linear"
data_x = np.random.random(100)

model = LinearModel()
with tf.Session(graph=model.graph) as sess:
    sess.run(tf.global_variables_initializer())
    ''' load pre-trained weight into model '''
    model.load_weight(sess, model_dir)
    ''' ################################## '''
    
    y = sess.run(model.varY, feed_dict={model.placeholderX: data_x})
    
    ans_y = lambda x: 0.5 * x + 0.3
    f, ax = plt.subplots()
    plt.scatter(data_x, y, label="predict", color="b", linewidth=5, alpha=0.5)
    plt.scatter(data_x, ans_y(data_x), label="truth", color="g")
    plt.legend(loc="best")
    plt.show()

## Observations
1. 上圖可知, 預測的答案跟實際答案是重疊的, 代表Model load success